In [24]:
import pandas as pd
import re
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import pickle
from datetime import datetime

### Reading KDD train file
Drop rows with null in any of the important columns

In [2]:
folder = 'data/kdd/2005-2006'

In [3]:
%%time
df_all = pd.read_csv('%s/algebra_2005_2006_train.txt' % folder, delimiter='\t')
print(df_all.shape)

(809694, 19)
CPU times: user 4.25 s, sys: 696 ms, total: 4.95 s
Wall time: 28 s


In [4]:
df_all

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
0,1,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG4-FIXED,1,3(x+2) = 15,2005-09-09 12:24:35.0,2005-09-09 12:24:49.0,2005-09-09 12:25:15.0,2005-09-09 12:25:15.0,40.0,NaN,40.0,0,2,3,1,[SkillRule: Eliminate Parens; {CLT nested; CLT...,1
1,2,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG4-FIXED,1,x+2 = 5,2005-09-09 12:25:15.0,2005-09-09 12:25:31.0,2005-09-09 12:25:31.0,2005-09-09 12:25:31.0,16.0,16.0,NaN,1,0,0,1,"[SkillRule: Remove constant; {ax+b=c, positive...",1~~1
2,3,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,1,2-8y = -4,2005-09-09 12:25:36.0,2005-09-09 12:25:43.0,2005-09-09 12:26:12.0,2005-09-09 12:26:12.0,36.0,NaN,36.0,0,2,3,1,"[SkillRule: Remove constant; {ax+b=c, positive...",2
3,4,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,1,-8y = -6,2005-09-09 12:26:12.0,2005-09-09 12:26:34.0,2005-09-09 12:26:34.0,2005-09-09 12:26:34.0,22.0,22.0,NaN,1,0,0,1,"[SkillRule: Remove coefficient; {ax+b=c, divid...",1~~1
4,5,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,2,-7y-5 = -4,2005-09-09 12:26:38.0,2005-09-09 12:28:36.0,2005-09-09 12:28:36.0,2005-09-09 12:28:36.0,118.0,118.0,NaN,1,0,0,1,"[SkillRule: Remove constant; {ax+b=c, positive...",3~~1
5,6,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,2,-7y = 1,2005-09-09 12:28:36.0,2005-09-09 12:28:43.0,2005-09-09 12:28:51.0,2005-09-09 12:28:51.0,15.0,NaN,15.0,0,1,0,1,"[SkillRule: Remove coefficient; {ax+b=c, divid...",2~~2
6,7,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,3,7y+4 = 7,2005-09-09 12:28:57.0,2005-09-09 12:29:09.0,2005-09-09 12:29:09.0,2005-09-09 12:29:09.0,12.0,12.0,NaN,1,0,0,1,"[SkillRule: Remove constant; {ax+b=c, positive...",4
7,8,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,3,7y = 3,2005-09-09 12:29:09.0,2005-09-09 12:29:14.0,2005-09-09 12:29:14.0,2005-09-09 12:29:14.0,5.0,5.0,NaN,1,0,0,1,[SkillRule: Remove positive coefficient; {ax/b...,1~~3
8,9,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,4,-5+9y = -6,2005-09-09 12:29:19.0,2005-09-09 12:29:31.0,2005-09-09 12:29:31.0,2005-09-09 12:29:31.0,12.0,12.0,NaN,1,0,0,1,"[SkillRule: Remove constant; {ax+b=c, positive...",5~~2
9,10,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,4,9y = -1,2005-09-09 12:29:31.0,2005-09-09 12:29:36.0,2005-09-09 12:29:36.0,2005-09-09 12:29:36.0,5.0,5.0,NaN,1,0,0,1,[SkillRule: Remove positive coefficient; {ax/b...,2~~4


In [5]:
df = df_all[['Anon Student Id', 'Correct First Attempt', 'Problem Hierarchy', 'Problem Name', 'Problem View', 
             'Step Name', 'KC(Default)', 'First Transaction Time',
             # 'Correct Transaction Time'
            ]]

In [6]:
df = df.dropna()

In [7]:
df.shape

(607025, 8)

### Preprocessing
Transform dataframe in a format understood by the BKT module

In [8]:
skipped = []
def get_kcs(row):
    text = row['KC(Default)'].replace(',','')
    kc_list = text.split('~~')
    kc = []
    for item in kc_list:
        # Option 1: with SkillRule prefix
        found_opt1 = re.findall(r'^\[SkillRule: ([\w ]+[ ,/?-]*[\w ]*);.*', item, flags=re.M)
        # Option 2: regular words (hyphenized or spaced)
        found_opt2 = re.findall(r'^([\w -]+).*', item, flags=re.M)
        # Option 3: get what couldn't be extracted from previous patterns
        found_opt3 = re.findall(r'^\[SkillRule: (.*);.*', item, flags=re.M)
        if found_opt1:
            kc.append(found_opt1[0])
        elif found_opt2:
            kc.append(found_opt2[0])
        elif found_opt3:
            kc.append(found_opt3[0])
        else:
            skipped.append(item)
    return kc

def get_steps(row):
    step = row['Problem Hierarchy'] + '_' + row["Problem Name"] + '_' + row['Step Name']
    return step

Separate KCs in a list and create an identifier name for the step

In [9]:
%%time
df['KC'] = df.apply(get_kcs, axis=1)
df['step'] = df.apply(get_steps, axis=1)

CPU times: user 36.4 s, sys: 464 ms, total: 36.9 s
Wall time: 36.9 s


In [10]:
%%time
kcs = []
for row in df['KC'].iteritems():
    kcs.extend(row[1])
kcs = list(set(kcs))
steps = df['step'].unique().tolist()

def create_data(row):
    question_id = steps.index(row['step'])
    return question_id

df['question_id'] = df.apply(create_data, axis=1)

CPU times: user 7min 11s, sys: 956 ms, total: 7min 12s
Wall time: 7min 12s


Save KCs

In [11]:
with open('%s/kdd_2005_2006_kcs_time.pkl' % folder, 'wb') as pklfile:
    pickle.dump(kcs, pklfile)
len(kcs)

110

Construct Q-Matrix

In [12]:
%%time
q_matrix = np.zeros((len(steps), len(kcs)))

for idx, item in df.iterrows():
    for kc in item['KC']:
        kc_col = kcs.index(kc)
        q_matrix[item['question_id'], kc_col] = 1
    
print(q_matrix.shape)

(176630, 110)
CPU times: user 1min 32s, sys: 168 ms, total: 1min 32s
Wall time: 1min 32s


Save data

In [13]:
%%time
with open('%s/kdd_2005_2006_df_time.pkl' % folder, 'wb') as pklfile:
    pickle.dump(df, pklfile)
with open('%s/kdd_2005_2006_q_matrix_time.pkl' % folder, 'wb') as pklfile:
    pickle.dump(q_matrix, pklfile)

CPU times: user 868 ms, sys: 284 ms, total: 1.15 s
Wall time: 1.15 s


In [32]:
%%time
# Data matrix
data = df[['Correct First Attempt', 'Anon Student Id', 'question_id', 'First Transaction Time']].values.tolist()
students = df['Anon Student Id'].unique().tolist()
len(students)

CPU times: user 516 ms, sys: 12 ms, total: 528 ms
Wall time: 535 ms


Transform time to datetime object

In [34]:
%%time
def create_time_object(x):
    x[3] = datetime.strptime(x[3], "%Y-%m-%d %H:%M:%S.0")
    
_ = list(map(create_time_object, data))

In [36]:
with open('%s/kdd_2005_2006_train_data_time.pkl' % folder, 'wb') as pklfile:
    pickle.dump(data, pklfile)
with open('%s/kdd_2005_2006_students_time.pkl' % folder, 'wb') as pklfile:
    pickle.dump(students, pklfile)